<a href="https://colab.research.google.com/github/masonnystrom/LS_Build_Week_Unit_1/blob/master/Maker_CDP_Data_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
import pandas_profiling

In [49]:
from google.colab import files
uploaded = files.upload()

Saving maker_vaults_activity_2019.csv to maker_vaults_activity_2019 (2).csv


In [50]:
"""
Attributes
DATE: The date associated with the set of metrics
LOCK: daily count of times when an owner adds PETH collateral.
GIVE: daily count of times when an owner transfers a CDP to a different owner.
EXIT: daily count of times when a user withdraws collateral by returning PETH to the vault in exchange for the proportional amount of ETH.
FREE: daily count of times when an owner removes PETH collateral.
OPEN: daily count of times when an empty CDP is created.
BITE: daily count of times when a vulnerable CDP is liquidated.
DRAW: daily count of times when a specified amount of Dai is issued.
JOIN: daily count of times when a user joins the collateral pool by depositing ETH in exchange for PETH (pooled ether).
SHUT: daily count of times when a CDP is closed, with all debt cleared, all collateral unlocked and records deleted.
WIPE: daily count of times when some portion of outstanding Dai debt is repayed.
"""

'\nAttributes\nDATE: The date associated with the set of metrics\nLOCK: daily count of times when an owner adds PETH collateral.\nGIVE: daily count of times when an owner transfers a CDP to a different owner.\nEXIT: daily count of times when a user withdraws collateral by returning PETH to the vault in exchange for the proportional amount of ETH.\nFREE: daily count of times when an owner removes PETH collateral.\nOPEN: daily count of times when an empty CDP is created.\nBITE: daily count of times when a vulnerable CDP is liquidated.\nDRAW: daily count of times when a specified amount of Dai is issued.\nJOIN: daily count of times when a user joins the collateral pool by depositing ETH in exchange for PETH (pooled ether).\nSHUT: daily count of times when a CDP is closed, with all debt cleared, all collateral unlocked and records deleted.\nWIPE: daily count of times when some portion of outstanding Dai debt is repayed.\n'

In [51]:
maker_vaults = pd.read_csv('maker_vaults_activity_2019.csv')
print(maker_vaults.shape)
maker_vaults.head()

(424, 11)


,date,actions_lock_total,actions_give_total,actions_exit_total,actions_free_total,actions_open_total,actions_bite_total,actions_draw_total,actions_join_total,actions_shut_total,actions_wipe_total
0,2019-01-15,167,21,92,92,29,2,144,168,4,99
1,2019-01-16,415,263,113,107,276,8,407,415,18,127
2,2019-01-17,408,265,133,112,291,18,395,414,20,123
3,2019-01-01,102,24,36,32,22,0,123,102,6,40
4,2019-01-09,100,24,40,35,36,1,113,100,4,41


In [0]:
# maker_vaults_profile = pandas_profiling.ProfileReport(maker_vaults)
# maker_vaults_profile

In [0]:
# need to sort by date cause it's off
maker_vaults['date'] = maker_vaults.sort_values(by=['date']).reset_index(drop=True)
maker_vaults['date'] = pd.to_datetime(maker_vaults['date'])

In [54]:
maker_vaults.head()

,date,actions_lock_total,actions_give_total,actions_exit_total,actions_free_total,actions_open_total,actions_bite_total,actions_draw_total,actions_join_total,actions_shut_total,actions_wipe_total
0,2019-01-01,167,21,92,92,29,2,144,168,4,99
1,2019-01-02,415,263,113,107,276,8,407,415,18,127
2,2019-01-03,408,265,133,112,291,18,395,414,20,123
3,2019-01-04,102,24,36,32,22,0,123,102,6,40
4,2019-01-05,100,24,40,35,36,1,113,100,4,41


In [0]:
# want vaults per month, and year
maker_vaults['month'] = maker_vaults['date'].dt.month
maker_vaults['year'] = maker_vaults['date'].dt.year

In [56]:
maker_vaults.isnull().sum()
# no nulls, thank you Ethereum 

date                  0
actions_lock_total    0
actions_give_total    0
actions_exit_total    0
actions_free_total    0
actions_open_total    0
actions_bite_total    0
actions_draw_total    0
actions_join_total    0
actions_shut_total    0
actions_wipe_total    0
month                 0
year                  0
dtype: int64

In [57]:
# looking at some of the data
# joins = when someone actually deposits ETH but counts for everytime
# lock = daily count of times when an owner adds PETH collateral.
# PETH which is then sent to the Vault and exchanged for Dai. The PETH is now locked until the DAI is repaid. 

print(maker_vaults['actions_open_total'].sum())
print(maker_vaults['actions_join_total'].sum())
print(maker_vaults['actions_lock_total'].sum())

142410
194561
193704


In [0]:
# going to drop columns where join is less tha

In [59]:
# maker_vaults that have been open and joined to filter out non-zero vaults
# and make sure we don't get repeat joins 
maker_vaults['opened_and_joined'] = (maker_vaults['actions_open_total'] & maker_vaults['actions_join_total'])
maker_vaults.head()

,date,actions_lock_total,actions_give_total,actions_exit_total,actions_free_total,actions_open_total,actions_bite_total,actions_draw_total,actions_join_total,actions_shut_total,actions_wipe_total,month,year,opened_and_joined
0,2019-01-01,167,21,92,92,29,2,144,168,4,99,1,2019,8
1,2019-01-02,415,263,113,107,276,8,407,415,18,127,1,2019,276
2,2019-01-03,408,265,133,112,291,18,395,414,20,123,1,2019,258
3,2019-01-04,102,24,36,32,22,0,123,102,6,40,1,2019,6
4,2019-01-05,100,24,40,35,36,1,113,100,4,41,1,2019,36


In [60]:
# Free = daily count of times when an owner removes PETH collateral
# Wipe = when someone repays a portion of Dai debt is repaid
# A user must free PETH before paying back Dai debt. 
print(maker_vaults['actions_free_total'].sum())
print(maker_vaults['actions_wipe_total'].sum())
print(maker_vaults['actions_shut_total'].sum())


48577
53221
18469


In [75]:
# check the creation of Maker vaults
# Open() is when daily count of times when an empty CDP is created.
plt.bar(maker_vaults2020['month'], maker_vaults2020['opened_and_joined'])

<BarContainer object of 58 artists>

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f6e27e26c80> (for post_execute):


ValueError: ignored

In [62]:
# lets find the total number of net maker vault joins
maker_vaults['net_opens'] = (maker_vaults['opened_and_joined'] - maker_vaults['actions_shut_total'])
maker_vaults['net_opens'].sum()

91154

In [0]:
plt.figure(figsize=(20,10), facecolor='whitesmoke')
plt.bar(maker_vaults['date'], maker_vaults['opened_and_joined'], color='green')
plt.bar(maker_vaults['date'], maker_vaults['actions_shut_total'], color ='red');

In [64]:
# check Maker Vault open to close net rate
print(maker_vaults['actions_shut_total'].sum())
print(maker_vaults['actions_open_total'].sum())

18469
142410


In [65]:
# lets examine only 2019 data
maker_vaults2019 = maker_vaults[maker_vaults['year']==2019]
print(maker_vaults2019.shape)
maker_vaults2019.head()

(366, 15)


,date,actions_lock_total,actions_give_total,actions_exit_total,actions_free_total,actions_open_total,actions_bite_total,actions_draw_total,actions_join_total,actions_shut_total,actions_wipe_total,month,year,opened_and_joined,net_opens
0,2019-01-01,167,21,92,92,29,2,144,168,4,99,1,2019,8,4
1,2019-01-02,415,263,113,107,276,8,407,415,18,127,1,2019,276,258
2,2019-01-03,408,265,133,112,291,18,395,414,20,123,1,2019,258,238
3,2019-01-04,102,24,36,32,22,0,123,102,6,40,1,2019,6,0
4,2019-01-05,100,24,40,35,36,1,113,100,4,41,1,2019,36,32


In [66]:
# make a 2020 DataFrame 
maker_vaults2020 = maker_vaults[maker_vaults['year']==2020]
print(maker_vaults2020.shape)
maker_vaults2020.head()

(58, 15)


,date,actions_lock_total,actions_give_total,actions_exit_total,actions_free_total,actions_open_total,actions_bite_total,actions_draw_total,actions_join_total,actions_shut_total,actions_wipe_total,month,year,opened_and_joined,net_opens
366,2020-01-01,51,35,65,37,11,2,64,52,34,41,1,2020,0,-34
367,2020-01-02,28,19,28,21,11,0,35,28,18,30,1,2020,8,-10
368,2020-01-03,35,26,41,27,12,0,52,36,20,22,1,2020,4,-16
369,2020-01-04,53,30,57,47,23,2,76,53,31,44,1,2020,21,-10
370,2020-01-05,98,26,31,33,26,0,94,97,4,25,1,2020,0,-4


In [0]:
# net opens/closers per day 2019
plt.figure(figsize=(15,12))
plt.bar(maker_vaults2019['date'], maker_vaults2019['net_opens'], color='#3DED97')
plt.show()

In [0]:
# net opens and shuts 2020
plt.figure(figsize=(12,8))
plt.bar(maker_vaults2020['date'], maker_vaults2020['net_opens'],color='#3DED97');

In [69]:
maker_vaults2019.sort_index(by=['actions_open_total'], ascending=False)[:5]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  """Entry point for launching an IPython kernel.


,date,actions_lock_total,actions_give_total,actions_exit_total,actions_free_total,actions_open_total,actions_bite_total,actions_draw_total,actions_join_total,actions_shut_total,actions_wipe_total,month,year,opened_and_joined,net_opens
287,2019-10-15,6815,6460,728,719,6475,7,7135,6818,545,904,10,2019,6146,5601
288,2019-10-16,5710,5361,892,872,5384,17,6057,5713,618,1020,10,2019,5120,4502
249,2019-09-07,4446,4214,361,345,4246,15,4552,4454,219,423,9,2019,4102,3883
291,2019-10-19,4248,3884,499,499,3951,1,4561,4248,353,562,10,2019,8,-345
289,2019-10-17,4000,3750,539,534,3745,73,4289,4000,398,678,10,2019,3744,3346


In [80]:
plt.scatter(5,10)

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f6e27e26c80> (for post_execute):


ValueError: ignored

In [70]:
print(maker_vaults2019['net_opens'].sum())
print(maker_vaults2020['net_opens'].sum())
maker_vaults['net_opens'].sum()
# interesting that there has been a net closers of Vaults in 2020.

60783
30371


91154

In [71]:
# times a Vault in danger was liquidated (need to find out what in danger means)
print(maker_vaults2019['actions_bite_total'].sum())
print(maker_vaults2020['actions_bite_total'].sum())

3367
2131


In [0]:
# installing plotly
import plotly.graph_objects as go

In [73]:
fig = go.Figure(data=go.Bar(y=[2, 3, 1]))
fig.show()

# Step 2: Check out the Second Data Set from Maker that shows top 1000 Accounts